# Two-dimensional compressible fluid dynamics example: Quadrants
In this example, we solve the Euler equations with initial data consisting of a different state in each quadrant of the unit square.  The following block of code will compute the solution at 40 timeframes and should take 1-2 minutes to run.

In [ ]:
from clawpack import pyclaw
from clawpack import riemann

claw = pyclaw.Controller()
claw.tfinal = 0.6
claw.num_output_times = 40

riemann_solver = riemann.euler_4wave_2D
claw.solver = pyclaw.ClawSolver2D(riemann_solver)
claw.solver.all_bcs = pyclaw.BC.extrap

grid_size = (300,300)
domain = pyclaw.Domain( (0.,0.), (1.,1.), grid_size)

claw.solution = pyclaw.Solution(claw.solver.num_eqn,domain)
gam = 1.4
claw.solution.problem_data['gamma']  = gam

# Set initial data
q = claw.solution.q
xx,yy = domain.grid.p_centers
l = xx<0.5; r = xx>=0.5; b = yy<0.5; t = yy>=0.5
q[0,...] = 2.*l*t + 1.*l*b + 1.*r*t + 3.*r*b
q[1,...] = 0.75*t - 0.75*b
q[2,...] = 0.5*l  - 0.5*r
q[3,...] = 0.5*q[0,...]*(q[1,...]**2+q[2,...]**2) + 1./(gam-1.)

claw.keep_copy = True       # Keep solution data in memory for plotting
claw.output_format = None   # Don't write solution data to file
claw.solver.dt_initial=1.e99
status = claw.run()

## Plotting: single frame with Matplotlib

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

frame = claw.frames[40]
density = frame.q[0,:,:]
(vx,vy) = np.gradient(density)
vs = np.sqrt(vx**2 + vy**2)
x, y = frame.state.grid.c_centers    

plt.pcolormesh(x, y, vs, cmap='RdBu')
plt.axis('image')

## Plotting: animation
Next we will plot an animation of all 40 frames, using matplotlib and Jake Vanderplas' JSAnimation code.

In [ ]:
from matplotlib import animation
import matplotlib.pyplot as plt
from clawpack.visclaw.JSAnimation import IPython_display
import numpy as np

fig = plt.figure(figsize=[4,4])

frame = claw.frames[0]
density = frame.q[0,:,:]
(vx,vy) = np.gradient(density)
vs = np.sqrt(vx**2 + vy**2)

x, y = frame.state.grid.c_centers    

# This essentially does a pcolor plot, but it returns the appropriate object
# for use in animation.  See http://matplotlib.org/examples/pylab_examples/pcolor_demo.html.
# Note that it's necessary to transpose the data array because of the way imshow works.
im = plt.imshow(vs.T, cmap='Greys', vmin=vs.min(), vmax=vs.max()/20,
           extent=[x.min(), x.max(), y.min(), y.max()],
           interpolation='nearest', origin='lower')

def fplot(frame_number):
    frame = claw.frames[frame_number]
    density = frame.q[0,:,:]
    (vx,vy) = np.gradient(density)
    vs = np.sqrt(vx**2 + vy**2)
    im.set_data(vs.T)
    return im,

In [ ]:
animation.FuncAnimation(fig, fplot, frames=len(claw.frames), interval=20)

## Plotting: zoomable, annotated plots with Plotly
Plotly is a web service that makes it possible (among many other things) to embed interactive plots in an IPython notebook.

In [ ]:
# This will only work if you have already installed the plotly package and logged into the plotly site.  First
# pip install plotly
# and then create an account and then do
# import plotly.tools as tls
# tls.set_credentials_file(username='username', api_key='key')

import plotly.plotly as py
import plotly.graph_objs as go

frame = claw.frames[40]
density = frame.q[0,:,:]
(vx,vy) = np.gradient(density)
vs = np.sqrt(vx**2 + vy**2)

vel2 = go.Heatmap(x=frame.grid.x.centers, y=frame.grid.y.centers, z=vs.T)

fig = go.Figure(data=[vel2])
fig['layout'].update(width=700, height=700, autosize=False)

# Note that it's necessary to transpose the data array because plotly's heatmap uses imshow.
py.iplot(fig,filename='quadrants')

## Plotting with VisClaw
To plot with VisClaw, we must first define a setplot function:

In [ ]:
def setplot(plotdata):    
    plotfigure = plotdata.new_plotfigure(name='Density', figno=0)
    plotaxes = plotfigure.new_plotaxes()
    plotaxes.title = 'Density'
    plotaxes.scaled = True      # so aspect ratio is 1
    plotitem = plotaxes.new_plotitem(plot_type='2d_schlieren')
    plotitem.plot_var = 0    
    return plotdata

Now we can plot a single frame as follows (most of this code will be unnecessary in the next version):

In [ ]:
from clawpack.visclaw import data
from clawpack.visclaw import frametools
plotdata = data.ClawPlotData()
plotdata.setplot = setplot
claw.plotdata = frametools.call_setplot(setplot,plotdata)

frame = claw.load_frame(40)
f=claw.plot_frame(frame)

VisClaw also has an interactive plotting loop capability.  However, that doesn't work well in the notebook, because no plot appears until you exit the loop, and then you only get the last plot.

In [ ]:
claw.plot()